In [1]:
import edt
import SimpleITK as sitk
from glob import glob
from pathlib import Path
import json
import pandas as pd
import os
import tqdm
import numpy as np

threshold = 0.5
test_vessel_dir = "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob"
external_vessel_dir = "/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_prob"

target_test_vessel_dir = (
    f"/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_edt_{str(threshold)}/"
)
# create if not exist
os.makedirs(target_test_vessel_dir, exist_ok=True)

target_external_vessel_dir = (
    f"/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_edt_{str(threshold)}"
)
os.makedirs(target_external_vessel_dir, exist_ok=True)

val_im_files = sorted(list(glob(f"{test_vessel_dir}/*_vessel.nii.gz")))
external_im_files = sorted(list(glob(f"{external_vessel_dir}/*_vessel.nii.gz")))

In [2]:
file_paths = val_im_files
save_dir = target_test_vessel_dir
# create dir if not exists
Path(save_dir).mkdir(parents=True, exist_ok=True)
for file in tqdm.tqdm(file_paths):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im_og = sitk.GetArrayFromImage(im_header)
    im = (im_og > threshold).astype(np.uint8)
    im_dist = edt.sdf(im, black_border=False, parallel=64)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid.replace("_vessel", ""))
    sitk.WriteImage(im_dist_header, save_file_path)

100%|██████████| 152/152 [1:38:32<00:00, 38.90s/it]


In [3]:
file_paths

['/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0001_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0002_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0003_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0004_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0005_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0006_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0007_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0008_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0009_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0010_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_prob/Ts0011_vessel.nii.gz',
 '/work/vig/Datasets/aneurysm/in

In [4]:
file_paths = external_im_files
save_dir = target_external_vessel_dir

for file in tqdm.tqdm(file_paths):
    scan_seriesuid = file.split("/")[-1]
    im_header = sitk.ReadImage(file)
    im = sitk.GetArrayFromImage(im_header)
    im = (im > threshold).astype(np.uint8)
    im_dist = edt.sdf(im, black_border=False, parallel=63)
    im_dist_header = sitk.GetImageFromArray(im_dist)
    im_dist_header.CopyInformation(im_header)
    save_file_path = os.path.join(save_dir, scan_seriesuid.replace("_vessel", ""))
    sitk.WriteImage(im_dist_header, save_file_path)

  0%|          | 0/138 [00:00<?, ?it/s]

100%|██████████| 138/138 [1:47:22<00:00, 46.69s/it]


In [ ]:
im_header = sitk.ReadImage(file)
im_dist = edt.sdf(im, black_border=False, parallel=22)

im_dist_saved = sitk.ReadImage(save_file_path)
im_dist_saved = sitk.GetArrayFromImage(im_dist_saved)

In [8]:
im_dist.shape, im_dist_saved.shape

((425, 507, 507), (425, 507, 507))

In [10]:
(im_dist - im_dist_saved).min()

0.0

In [ ]:
header = sitk.ReadImage(
    "/work/vig/Datasets/aneurysm/internal_train/crop_0.4_vessel_edt/Tr0188.nii.gz"
)